In [2]:
import pandas as pd
import ccxt
from binance.client import Client
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import talib
from datetime import datetime, timedelta
from backtesting import Strategy, Backtest
from backtesting.lib import crossover
from backtesting.test import SMA


In [4]:
API_KEY = 'your key'
API_SECRET = 'your key'
client = Client(API_KEY,API_SECRET)

In [6]:
#Creating DataFrame using Binance, data of BTC/USDT

# exchange = ccxt.binance()
symbol = 'ETH/USDT'
timeframe = '4H'
ohlcv = client.get_historical_klines('ETHUSDT',Client.KLINE_INTERVAL_4HOUR,'13 Aug, 2017','25 Apr, 2022')
to_df = pd.DataFrame(ohlcv)
clean_df = to_df.drop(columns=[0,6,7,8,9,10,11])

clean_df.columns = ['Open','High','Low','Close','Volume']
clean_df['Open'] = pd.to_numeric(clean_df['Open'])
clean_df['High'] = pd.to_numeric(clean_df['High'])
clean_df['Low'] = pd.to_numeric(clean_df['Low'])
clean_df['Close'] = pd.to_numeric(clean_df['Close'])
clean_df['Volume'] = pd.to_numeric(clean_df['Volume'])

data_is=clean_df.head(6768) #66% de los datos
data_oos = clean_df.tail(3487) #33% de los datos


In [7]:
clean_df.head(10)

,Open,High,Low,Close,Volume
0,301.13,307.96,298.00,307.96,1561.95305
1,307.95,312.00,307.00,308.95,1177.71088
2,308.95,310.51,303.56,307.06,1882.05267
3,307.74,312.18,298.21,301.60,1208.05192
4,301.60,310.85,299.01,302.00,1200.94182
5,302.00,306.47,295.19,301.40,1690.58319
6,301.40,309.49,298.41,306.24,1732.52616
7,306.24,311.79,303.22,307.78,1213.01283
8,307.78,310.29,296.32,298.80,1358.99471
9,298.80,301.11,287.37,293.83,1620.71499


In [53]:
class SMACross(Strategy):
    mm1 = 8
    mm2 = 20
    mm3 = 30
    mm4 = 200

    def init(self):
        super().init()
        self.sma8 = self.I(SMA, self.data.Close, self.mm1)
        self.sma20 = self.I(SMA, self.data.Close, self.mm2)
        self.sma30 = self.I(SMA, self.data.Close, self.mm3)
        self.sma200 = self.I(SMA, self.data.Close, self.mm4)
    
    def next(self):
        super().next()
        self.price = clean_df['Close'].iloc[0]
    
        if not self.position:
            if self.sma20 > self.sma30 and self.price >= self.sma20 or \
                self.sma20 > self.sma200 and self.price >= self.sma20:
                self.buy()
            elif self.sma20 < self.sma30 and self.price < self.sma20 or \
                self.sma20 < self.sma200 and self.price < self.sma30:
                self.sell()
        else:
            if self.position.is_long and self.sma8 < self.sma20 or \
            self.price == self.sma200:
                self.position.close()
            elif self.position.is_short and self.sma8 > self.sma20 or \
            self.price == self.sma200:
                self.position.close()

bt = Backtest(clean_df,SMACross, cash = 10000, trade_on_close=False)
output = bt.run()
print(output)
bt.plot()


C:\Users\MI PC\AppData\Local\Temp\ipykernel_4072\2241283795.py:33: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(clean_df,SMACross, cash = 10000, trade_on_close=False)


Start                                     0.0
End                                   10254.0
Duration                              10254.0
Exposure Time [%]                   64.446611
Equity Final [$]                      8400.49
Equity Peak [$]                      46988.56
Return [%]                           -15.9951
Buy & Hold Return [%]              833.754384
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -82.728285
Avg. Drawdown [%]                   -9.335523
Max. Drawdown Duration                 3578.0
Avg. Drawdown Duration                178.625
# Trades                               1384.0
Win Rate [%]                        47.182081
Best Trade [%]                      40.762125
Worst Trade [%]                    -18.021183
Avg. Trade [%]                    

Row(id='23279', ...)